<a href="https://colab.research.google.com/github/AhlamBashiti1/MedCUI_ML_Project/blob/main/BioBertEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch pandas

In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from google.colab import files


In [ ]:
#  Load CSV File
uploaded = files.upload()  #  cui_mapping.csv f
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model.eval()


In [ ]:
def get_embedding(text: str) -> np.ndarray:
    """Generate BioBERT embedding for a given text using CLS token."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()


In [ ]:
def get_embedding(text: str) -> np.ndarray:
    """Generate BioBERT embedding for a given text using CLS token."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    # Average all token embeddings
    #return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


embeddings = []
for name in tqdm(df["Canonical name"], desc="Generating Embeddings"):
    embeddings.append(get_embedding(str(name)))


In [ ]:

# Save as .npy file
np.save("biobert_embeddings.npy", np.array(embeddings))

# Attach embeddings to DataFrame
df["embedding"] = [embedding.tolist() for embedding in embeddings]
df.to_csv("cui_with_embeddings.csv", index=False)

# results
files.download("biobert_embeddings.npy")
files.download("cui_with_embeddings.csv")
